# Bottom Data for 2012-2013 
#### includes data from upriver, downriver, minilander, and midestuary 


In [ ]:
import pandas as pd
import scipy.io as sio
import numpy as np
import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import re

def Array_To_DataFrame(matDF, varName) :
    npArray = np.array([])
    for i in range (matDF[varName].shape[0]) :
        npArray = np.concatenate((npArray, matDF[varName][i].flatten()))
    return npArray


#read in surface data from each of the locations
downDF = pd.read_csv('downriver_2012_2013_processed.csv', parse_dates = ['DN'])
upDF = pd.read_csv('upriver_2012_2013_processed.csv', parse_dates = ['DN'])
midDF = pd.read_csv('midestuary_2012_2013_processed.csv', parse_dates = ['DN'])
combinedDF = pd.read_csv('ocean_temperature12_13.csv', parse_dates = ['DN', 'dayDN'])
riverCombinedDF = pd.read_csv('river_temperature12_13.csv', parse_dates = ['DN', 'dayDN'])
miniImportDF = pd.read_csv('minilander_2012_2013_processed.csv', parse_dates = ["DN"])


miniDF = pd.merge_asof(downDF, miniImportDF, on = 'DN')[["DN", "Temperature", "Salinity"]]
oceanTempDF = pd.merge_asof(downDF, combinedDF, left_on = 'DN', right_on = 'dayDN', direction = 'nearest')[["DN_x", "BottomTemp_y", "Temperature", "BottomSalCorrected_y", "Salinity", "dailyAverage", "AvgEstimatedTemp", "oceanSalEstimate"]]
oceanTempDF["AvgEstimatedTemp"] = oceanTempDF["AvgEstimatedTemp"].where(~oceanTempDF["AvgEstimatedTemp"].duplicated(), pd.NA)
oceanTempDF["oceanSalEstimate"] = oceanTempDF["oceanSalEstimate"].where(~oceanTempDF["oceanSalEstimate"].duplicated(), pd.NA)
oceanTempDF = oceanTempDF.interpolate()
riverTempDF = pd.merge_asof(downDF, riverCombinedDF, left_on = 'DN', right_on = 'dayDN', direction = 'nearest')[["DN_x","surfaceRiver", "bottomRiver", "EstimatedTemp"]]
riverTempDF["EstimatedTemp"] = riverTempDF["EstimatedTemp"].where(~riverTempDF["EstimatedTemp"].duplicated(), pd.NA)
riverTempDF = riverTempDF.interpolate()
oceanTempDF = oceanTempDF.rename(columns={"DN_x": "DN", "BottomTemp_y": "BottomTemp", "Temperature" : "MiniTemp", "BottomSalCorrected_y" : "BottomSalCorrected", "Salinity" : "MiniSal", "dailyAverage" : "Stonewall", "AvgEstimatedTemp" : "OceanTemp"})
#reading in flow data
flowDF = pd.read_csv('2012_2013alsea.csv')
flowDF['fRate'] = flowDF['max_va'] * 0.0283168
#create a new column for flow dates
for index, row in flowDF.iterrows():
    calcDate = pd.Timestamp(year=int(row['begin_yr']), month=int(row['month_nu']), day=int(row['day_nu']))
    flowDF.loc[index, 'DN'] = calcDate

tmsFlow = flowDF['DN']

# Now we need to refill beginning and end of midDF, as it's time scale starts later than others and ends earlier than others
preTimestamp = pd.Timestamp('2012-12-12 17:15:00')
postTimestamp = pd.Timestamp('2013-05-21 19:00:00')
firstMidMark = midDF['DN'][0]
lastMidMark = midDF['DN'][len(midDF['DN'])-1]
while (preTimestamp < firstMidMark):
    dataMidDF = pd.DataFrame({'DN' : [preTimestamp], 'SurfaceSalCorrected' : [np.nan], 'BottomSalCorrected' : [np.nan], 'BottomSalBad' : [True], 'SurfaceTemp' : [np.nan], 'BottomTemp' : [np.nan], 'BottomTempBad' : [True], 'BottomPressure' : [np.nan]})
    midDF = pd.concat([midDF, dataMidDF])
    preTimestamp += datetime.timedelta(minutes = 5)
while postTimestamp > lastMidMark:
    lastMidMark += datetime.timedelta(minutes = 5)
    dataMidDF = pd.DataFrame({'DN' : [lastMidMark], 'SurfaceSalCorrected' : [np.nan], 'BottomSalCorrected' : [np.nan], 'BottomSalBad' : [True], 'SurfaceTemp' : [np.nan], 'BottomTemp' : [np.nan], 'BottomTempBad' : [True], 'BottomPressure' : [np.nan]})
    midDF = pd.concat([midDF, dataMidDF])
midDF = midDF.sort_values(['DN'], ascending = [True])

riverTempDF["riverSalinity"] = 0
tempFigLabels = ['Downriver', 'Upriver', 'Midestuary', 'BottomEstimate', 'MiniEstimate', 'Stonewall', "surfaceRiver", "bottomRiver", "MiniTemp"]
tempfig = px.line(x = upDF['DN'], y = [downDF['BottomTemp'], upDF['BottomTemp'], midDF['BottomTemp'], oceanTempDF["BottomTemp"], oceanTempDF["MiniTemp"],oceanTempDF["Stonewall"], riverTempDF["surfaceRiver"], riverTempDF["bottomRiver"], miniDF["Temperature"]], color_discrete_sequence= ["black", "cyan", "blue", "purple", "orange", "red", "magenta", "olive", "dodgerblue"], title = "Temperature")
for idx in range(len(tempFigLabels)):
    tempfig.data[idx].name = tempFigLabels[idx]
    tempfig.data[idx].hovertemplate = 'variable=' + tempFigLabels[idx] + '<br>x=%{x}<br>value=%{y}<extra></extra>'
    tempfig.data[idx].legendgroup = tempFigLabels[idx]
tempfig.update_traces(patch={"line": {"dash": "dot"}}, selector={'name' : 'BottomEstimate'})
tempfig.update_traces(patch={"line": {"dash": "dot"}}, selector={'name' : 'MiniEstimate'})
tempfig.update_traces(patch={"line": {"dash": "dot"}}, selector={'name' : 'Stonewall'})
tempfig.update_traces(patch={"line": {"dash": "dot"}}, selector={'name' : 'surfaceRiver'})
tempfig.update_traces(patch={"line": {"dash": "dot"}}, selector={'name' : 'bottomRiver'})
tempfig.update_traces(patch={"line": {"width": 2}}, selector={'name' : 'Downriver'})
tempfig.update_traces(patch={"line": {"width": 2}}, selector={'name' : 'Upriver'})
tempfig.update_traces(patch={"line": {"width": 2}}, selector={'name' : 'Midestuary'})
tempfig.update_traces(patch={"line": {"width": 2}}, selector={'name' : 'MiniTemp'})
tempfig.add_scattergl(x= oceanTempDF['DN'], y = oceanTempDF["OceanTemp"], line={'color': 'green', 'width' : 2}, name = "Ocean Temperature")
tempfig.add_scattergl(x= upDF['DN'], y = riverTempDF["EstimatedTemp"], line={'color': 'gray', 'width' : 2}, name = "River Temperature")

Bottom1213DataDF = pd.DataFrame({'DN' : upDF.reset_index(drop=True)['DN'], 'midestuary' : midDF.reset_index(drop=True)['BottomTemp'], 'upriver' : upDF.reset_index(drop=True)['BottomTemp'], 'downriver' : downDF.reset_index(drop=True)['BottomTemp']})
Bottom1213DataDF.to_csv("bottom_data_12-13.csv", index = False)




salFigLabels = ['Downriver', 'Upriver', 'Midestuary', "MiniSal", "River Salinity", "Ocean Salinity"]
salfig = px.line(x = downDF['DN'], y = [downDF['BottomSalCorrected'], upDF['BottomSalCorrected'], midDF['BottomSalCorrected'], miniDF['Salinity'], riverTempDF["riverSalinity"], oceanTempDF["oceanSalEstimate"]], color_discrete_sequence= ["black", "cyan", "blue", "green", "orange", "red"], title = "Salinity")
for idx in range(len(salFigLabels)):
    salfig.data[idx].name = salFigLabels[idx]
    salfig.data[idx].hovertemplate = 'variable=' + salFigLabels[idx] + '<br>x=%{x}<br>value=%{y}<extra></extra>'
    salfig.data[idx].legendgroup = salFigLabels[idx]

flowfig = px.line(x = tmsFlow, y = [flowDF['fRate']], color_discrete_sequence= ["green"], title = "Flow Rate")
flowFigLabel = ['Flow Rate']
for idx in range(len(flowFigLabel)):
    flowfig.data[idx].name = flowFigLabel[idx]
    flowfig.data[idx].hovertemplate = 'variable=' + flowFigLabel[idx] + '<br>x=%{x}<br>value=%{y}<extra></extra>'
    flowfig.data[idx].legendgroup = flowFigLabel[idx]

tempfig.update_layout(title=dict(text= "Bottom Temperature for 2012-2013", font=dict(size=25)))
tempfig.update_xaxes(tickangle=30)
tempfig.update_xaxes(rangeslider_visible=True)
tempfig.update_xaxes(range = [pd.Timestamp('2012-12-12'),pd.Timestamp('2013-05-21')])

salfig.update_layout(title=dict(text= "Bottom Salinity for 2012-2013", font=dict(size=25)))
salfig.update_xaxes(tickangle=30)
salfig.update_xaxes(rangeslider_visible=True)
salfig.update_xaxes(range = [pd.Timestamp('2012-12-12'),pd.Timestamp('2013-05-21')])

flowfig.update_layout(title=dict(text= "Flow Rate for 2012-2013", font=dict(size=25)))
flowfig.update_xaxes(tickangle=30)
flowfig.update_xaxes(rangeslider_visible=True)
flowfig.update_xaxes(range = [pd.Timestamp('2012-12-12'),pd.Timestamp('2013-05-21')])

tempfig.update_layout(xaxis_title="Date", yaxis_title="Temperature (°C)", legend_title="Locations")
salfig.update_layout(xaxis_title="Date", yaxis_title="Salinity", legend_title="Locations")
flowfig.update_layout(xaxis_title="Date", yaxis_title="Cubic Meters per Second",legend_title="Locations")

tempfig.show()
salfig.show() 
flowfig.show() 